In [1]:
from actions import ActionsWorkflow, ActionsJobs, ActionsArtifacts
import LogExtractor as extractor

from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Table, TableStyle, Spacer, ListFlowable, ListItem
from datetime import datetime

import pandas as pd
import numpy as np


## Retrieving test data and processing 

In [2]:
repo_path = 'MagaluCloud/s3-specs'
query_size = 10


In [3]:

workflow = ActionsWorkflow(repository=repo_path, query_size=query_size)
jobs = ActionsJobs(repo_path, workflow)

all_workflows_jobs = pd.DataFrame()

for id in workflow.df['databaseId']:
    tmp = jobs.get_jobs(id)
    all_workflows_jobs = pd.concat([all_workflows_jobs, tmp])

DataFrame successfully saved to ./bin/actionsWorflow.parquet


In [4]:
artifacts = ActionsArtifacts(repository=repo_path)
all_tests_df = pd.DataFrame()
all_times_df = pd.DataFrame()
all_failures_df = pd.DataFrame()

for path in artifacts.paths:
    artifact = extractor.PytestArtifactLogExtractor(path)
    pytest_tests_status	, pytest_run_times, pytest_failures_errors = artifact.log_to_df()
    all_tests_df = pd.concat([all_tests_df, pytest_tests_status])
    all_times_df = pd.concat([all_times_df, pytest_run_times])
    all_failures_df = pd.concat([all_failures_df, pytest_failures_errors])

[' short test summary info ', 'FAILED versioning_cli_test.py::test_delete_object_with_versions[mgc object-storage objects delete {bucket_name}/{object_key} --no-confirm-] - AssertionError: Command failed with error: Error: (NoSuchBucket) 404 Not Found - The specified bucket does not exist. (request-id: adb484ed-e14e-4f6c-a361-1386fc0dcf02)', '  ', 'assert 1 == 0', " +  where 1 = CompletedProcess(args=['mgc', 'object-storage', 'objects', 'delete', 'test-versioned-bucket-with-one-object69f5fe/test-object.txt', '--no-confirm'], returncode=1, stdout='', stderr='Error: (NoSuchBucket) 404 Not Found - The specified bucket does not exist. (request-id: adb484ed-e14e-4f6c-a361-1386fc0dcf02)\\n').returncode", 'FAILED versioning_cli_test.py::test_delete_bucket_with_objects_with_versions[mgc object-storage buckets delete {bucket_name} --no-confirm --recursive --raw-the bucket may not be empty] - assert \'the bucket may not be empty\' in \'WARN\\tgithub.com/MagaluCloud/magalu/mgc/sdk/static/object_s

ValueError: 5 columns passed, passed data had 2 columns

In [ ]:
# Trocar databaseId por jobId
display(all_tests_df[all_tests_df['arguments'].notna()])
display(all_times_df[all_times_df['databaseId'].notna()])
display(all_failures_df[all_failures_df['databaseId'] != None])

,status,category,arguments,databaseId
pytest_tests_status,,,,
test_acl_operations,PASSED,"ACL=authenticated-read,Method=list_multipart_u...",],13269149128
test_acl_operations,PASSED,"ACL=public-read,Method=list_objects_v2,Context...",],13269149128
test_acl_operations,PASSED,"ACL=authenticated-read,Method=delete_object,Co...",],13269149128
test_acl_operations,PASSED,"ACL=public-read,Method=list_object_versions,Co...",],13269149128
test_acl_operations,PASSED,"ACL=authenticated-read,Method=put_object,Conte...",],13269149128
...,...,...,...,...
test_acl_operations,PASSED,"ACL=private,Method=get_object,Context=object,E...",],13269265723
test_acl_operations,PASSED,"ACL=private,Method=put_object,Context=bucket,E...",],13269265723
test_put_invalid_bucket_policy,PASSED,"{\n""Version"":""2012-10-18"",\n""Statement"":[\n{\n...",\n}\n-MalformedJSON],13269265723


,total,num,avg,min,durationType,databaseId
pytest_run_times,,,,,,
test_create_same_bucket,1.063,1,1.063,1.063,teardown duration,13269265728
test_create_bucket,0.878,1,0.878,0.878,teardown duration,13269265728
test_create_same_bucket,1.349,1,1.349,1.349,setup duration,13269265728
test_boto_list_buckets,0.266,1,0.266,0.266,setup duration,13269265728
test_create_bucket,0.094,1,0.094,0.094,setup duration,13269265728
...,...,...,...,...,...,...
test_delete_bucket_with_objects_with_versions,5.903,4,1.705,0.618,call duration,13269265723
versioned_bucket_with_one_object,46.476,8,5.772,5.330,fixture duration,13269265723
s3_client,1.409,8,0.074,0.067,fixture duration,13269265723


,category,arguments,error,error_details,databaseId
pytest_failures_errors,,,,,
None,None,None,None,None,None
None,None,None,None,None,None
None,None,None,None,None,None
None,None,None,None,None,13269265728
None,None,None,None,None,13269265728
None,None,None,None,None,13269265728
None,None,None,None,None,None
None,None,None,None,None,None
None,None,None,None,None,None


## Creating pdf

In [ ]:
total_times = pd.Series(dict(map(lambda t: (t, sum(pytest_run_times[pytest_run_times.index == t].get('total'))), pytest_tests_status.category.unique())))
avg_time_test  = pd.Series(dict(map(lambda t: (t, sum(pytest_run_times[pytest_run_times.index == t].get('avg'))), pytest_tests_status.category.unique())))
min_test_time  = pd.Series(dict(map(lambda t: (t, sum(pytest_run_times[pytest_run_times.index == t].get('min'))), pytest_tests_status.category.unique())))
total_nums = pytest_tests_status['category'].value_counts()
total_passed = pytest_tests_status[pytest_tests_status.index.values != 'PASSED'].set_index('category').index.value_counts()
time_count_df = pd.concat([total_passed, total_nums - total_passed, total_nums, min_test_time, avg_time_test, total_times], axis=1)
time_count_df.columns = ['num_passed', 'num_failed', 'total_runs', 'min_test_time', 'avg_test_time', 'total_duration']

report_df = pd.DataFrame()
report_df['name'] = pytest_tests_status['category'].unique()
report_df = report_df.set_index('name')

report_df = pd.concat([report_df, time_count_df], axis=1)
report_df

,num_passed,num_failed,total_runs,min_test_time,avg_test_time,total_duration
mgcobject-storageobjectsdelete{bucket_name}/{object_key}--no-confirm-,1.0,0.0,1.0,0,0,0
rclonedelete{profile_name}:{bucket_name}/{object_key}-,1.0,0.0,1.0,0,0,0
aws--profile{profile_name}s3rms3://{bucket_name}/{object_key}-delete:s3://{bucket_name}/{object_key}\n,1.0,0.0,1.0,0,0,0
mgcobject-storagebucketsdelete{bucket_name}--no-confirm--recursive--raw-thebucketmaynotbeempty,1.0,0.0,1.0,0,0,0
rclonermdir{profile_name}:{bucket_name}-BucketNotEmpty,1.0,0.0,1.0,0,0,0
aws--profile{profile_name}s3rbs3://{bucket_name}-BucketNotEmpty,1.0,0.0,1.0,0,0,0
None,NaN,NaN,NaN,0,0,0


In [ ]:
failed_details_df = pytest_failures_errors.set_index('category').drop(columns=['arguments', 'databaseId'])
df = failed_details_df.reset_index()
df = report_df.reset_index().round(3)
df

,index,num_passed,num_failed,total_runs,min_test_time,avg_test_time,total_duration
0,mgcobject-storageobjectsdelete{bucket_name}/{o...,1.0,0.0,1.0,0,0,0
1,rclonedelete{profile_name}:{bucket_name}/{obje...,1.0,0.0,1.0,0,0,0
2,aws--profile{profile_name}s3rms3://{bucket_nam...,1.0,0.0,1.0,0,0,0
3,mgcobject-storagebucketsdelete{bucket_name}--n...,1.0,0.0,1.0,0,0,0
4,rclonermdir{profile_name}:{bucket_name}-Bucket...,1.0,0.0,1.0,0,0,0
5,aws--profile{profile_name}s3rbs3://{bucket_nam...,1.0,0.0,1.0,0,0,0
6,None,NaN,NaN,NaN,0,0,0


Blocos que vão existir

Topo: Contendo Informações básicas do relatório e se possível alguns campos em branco e também a data de quando o código foi executado

Texto: Informações gerais do número de acertos e erros

Tabela: Contem o dataframe report_df, mas estilizado

Gráficos: fica pra dps

In [ ]:

# Function to create PDF
def create_pdf(df):
    # A4 size dimensions
    width, height = A4

    # Set 10% margin
    margin = 0.1 * width

    # Create PDF with margins
    doc = SimpleDocTemplate("report_v0.pdf", pagesize=A4,
                            leftMargin=margin, rightMargin=margin, topMargin=0.1*height, bottomMargin=0.1*height)

    # Styles
    styles = getSampleStyleSheet()
    heading_style = styles['Heading1']
    normal_style = styles['Normal']
    normal_style.alignment = 0  # 0 for left alignment

    bold_style = ParagraphStyle(
        name="Bold",
        parent=normal_style,
        fontName="Helvetica-Bold",
        fontSize=12
    )

    # Create the story (content) for the PDF
    story = []

    # Add title with fields
    story.extend(create_title(heading_style,normal_style))

    # Add each section to the story
    story.extend(create_execution_summary(df, normal_style, bold_style))
    story.extend(create_detailed_results(df, normal_style, bold_style, width, margin))
    story.extend(create_errors_summary(normal_style, bold_style, width, margin))

    # Build PDF
    doc.build(story)

# 
def create_title(heading_style, normal_style):
    # Initialize the story list
    story = []

    # Get current date and time
    agora = datetime.now()
    horario_dia = agora.strftime("%d/%m/%Y %H:%M:%S")

    # Create the title
    title_text = "Sumário de Resultados dos Testes"
    title_paragraph = Paragraph(f"<b>{title_text}</b>", heading_style)

    # Create the formatted text for the execution date on the right side
    right_date_style = ParagraphStyle(
        "RightDateStyle", parent=normal_style, alignment=2, fontSize=10
    )
    date_paragraph = Paragraph(horario_dia, right_date_style)

    # Add title and date to the story as separate elements
    story.append(title_paragraph)
    story.append(date_paragraph)

    # Create the formatted text for the execution date, system version, and environment
    execution_paragraph = Paragraph(f"Data da Execução: ", normal_style)
    version_paragraph = Paragraph("Versão do Sistema: ", normal_style)
    environment_paragraph = Paragraph("Ambiente: ", normal_style)

    # Add other paragraphs to the story
    story.append(execution_paragraph)
    story.append(Spacer(1, 12))  # Spacer between execution and version
    story.append(version_paragraph)
    story.append(Spacer(1, 12))  # Spacer between version and environment
    story.append(environment_paragraph)

    story.append(Spacer(1, 24))  # Add space at the end

    # Return the complete story
    return story



# Function to create execution summary table with bullet points
def create_execution_summary(df, normal_style, bold_style):
    story = []
    story.append(Paragraph("Resumo Geral", bold_style))
    story.append(Spacer(1, 6))

    # Criando a lista de resumo corretamente
    summary_data = {
        'Total de Testes:': df['total_runs'].sum(),
        'Testes Bem-Sucedidos:': df['num_passed'].sum(),
        'Testes com Falha:': df['num_failed'].sum(),
        'Tempo Mínimo de Execução:': f"{df['min_test_time'].min():.2f} s",
        'Tempo Médio de Execução:': f"{df['avg_test_time'].mean():.2f} s",
        'Duração Total dos Testes:': f"{df['total_duration'].sum():.2f} s"
    }

    # Criando a lista com bullet points
    bullet_points = [
        ListItem(Paragraph(f"<b>{key}</b> {value}", normal_style), leftIndent=20, spaceAfter=6)
        for key, value in summary_data.items()
    ]

    # Criando o ListFlowable
    list_flowable = ListFlowable(bullet_points, bulletType='bullet', leftIndent=20)

    # Adicionando ao relatório
    story.append(list_flowable)
    story.append(Spacer(1, 24))

    return story

# Function to create detailed results table
def create_detailed_results(df, normal_style, bold_style, width, margin):
    story = []
    story.append(Paragraph("Detalhamento dos Testes", bold_style))
    story.append(Spacer(1, 12))
    df_renamed = df.copy()  # Create a copy of the DataFrame
    df_renamed.columns = [
        'Nome do Teste', 
        'Testes Bem-Sucedidos', 
        'Falhas', 
        'Execuções', 
        'Tempo Mínimo de Execução', 
        'Tempo Médio', 
        'Duração Total'
    ]

    # Dropping the specified columns
    df_renamed = df_renamed.drop(columns=['Testes Bem-Sucedidos', 'Tempo Mínimo de Execução'])

    df_renamed['Tempo Médio'] = df_renamed['Tempo Médio'].astype(str) + ' sec'
    df_renamed['Duração Total'] = df_renamed['Duração Total'].astype(str) + ' sec'

    # Prepare the detailed data for the table
    detailed_tests_data = [df_renamed.columns.tolist()]  # Add header
    detailed_tests_data.extend(
        [[Paragraph(str(value), normal_style) for value in row] for row in df_renamed.values.tolist()]
    )

    # Calculate available width after applying margins
    available_width = width - 2 * margin  # Subtracting left and right margins

    # Define column proportions
    proportions = [0.3, 0.15, 0.15, 0.15, 0.2]  # Example proportions

    total_proportion = sum(proportions)
    if total_proportion > 1:
        proportions = [p / total_proportion for p in proportions]  # Scale proportions to fit within 1

    # Calculate column widths based on the available width
    col_widths = [available_width * p for p in proportions]

    # Create the table
    detailed_table = Table(detailed_tests_data, colWidths=col_widths)
    detailed_table.setStyle(TableStyle([('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                                        ('GRID', (0, 0), (-1, -1), 0.5, colors.black)]))
    story.append(detailed_table)
    story.append(Spacer(1, 24))

    return story

# Function to create errors summary as a numbered list
def create_errors_summary(normal_style, bold_style, width, margin):
    story = []
    story.append(Paragraph("Resumo dos Erros", bold_style))
    story.append(Spacer(1, 12))

    # Define um estilo menor para os números da lista
    small_number_style = ParagraphStyle(
        "small_number_style",
        parent=normal_style,
        fontSize=10  # Tamanho menor para os números
    )

    # Lista de erros
    errors_list = [
        "Timeout na comunicação com a API (Testes: TC_002)",
        "Falha de autenticação com banco de dados (Testes: TC_007)",
        "Erro de processamento de dados em alta carga (Testes: TC_015, TC_016)"
    ]

    # Criando itens da lista com mais espaço entre eles
    numbered_items = [
        ListItem(Paragraph(error, normal_style), leftIndent=margin, spaceAfter=10)
        for error in errors_list
    ]

    # Criando a lista numerada
    numbered_list = ListFlowable(
        numbered_items,
        bulletType='1',  # Define como lista numerada (1., 2., 3.)
        bulletFontSize=10,  # Define tamanho menor para os números
        leftIndent=margin*0.5  # Ajustando a indentação para alinhar com as margens
    )

    story.append(numbered_list)
    story.append(Spacer(1, 24))

    return story

# Gerar o PDF
create_pdf(df)
